# Crashes Near Point

Find the crash incident_id for a specific crash given its location and time. 

In [1]:
import folium
import datetime
import pandas as pd
from folium.plugins import HeatMap

import os
os.chdir('..')
from scripts.crash_data_analysis import CrashDataAnalysis

In [2]:
cda = CrashDataAnalysis()
# df = cda.crash_dataframe(verbose=True)

In [3]:
# geocoded_location = 'S Zuni St and W Arkansas Ave, Denver, CO'
# geocoded_location = 'Godsman Elementary School, Denver, CO'
# geocoded_location = 'W 26th Ave and Tennyson St, Denver, CO'
# geocoded_location = 'E 1st Ave and N Gilpin St, Denver, CO'

# crash occurred within this distance of the given latitude and longitude
radius_miles = 0.5

In [4]:
# lat_lon = cda.geocode_location(geocoded_location)
# lat_lon

In [12]:
geocoded_location = 'E 1st Ave and N Gilpin St, Denver, CO'
lat_lon = [39.71824953362511, -104.96668328236497] # 'E 1st Ave and N Gilpin St, Denver, CO'

In [6]:
query = f"""
select
*
, reported_date at time zone 'America/Denver' as reported_date_local
, ST_Distance(
    geom_denver, ST_Transform(ST_SetSRID(ST_MakePoint({lat_lon[1]},{lat_lon[0]}), 4326), 3502)
) as distance_feet

from crashes

where geo_lon is not null
and ST_Distance(
    geom_denver, ST_Transform(ST_SetSRID(ST_MakePoint({lat_lon[1]},{lat_lon[0]}), 4326), 3502)
    ) < {radius_miles * 5280}

order by reported_date
"""
# and extract(year from reported_date) >= 2022

nearby = pd.read_sql(query, cda.conn)

nearby['reported_date_local'] = nearby['reported_date_local'].dt.tz_localize('America/Denver')

In [7]:
nearby.groupby(nearby.reported_date.dt.year).agg(
    crashes=('incident_id', 'count')
    , fatalities=('fatality', 'sum')
    , sbi=('sbi', 'sum')
    , pedestrian_ind=('pedestrian_ind', 'sum')
    , bicycle_ind=('bicycle_ind', 'sum')
).astype(int) #.to_clipboard()

,crashes,fatalities,sbi,pedestrian_ind,bicycle_ind
reported_date,,,,,
2013,141,0,0,4,2
2014,156,1,1,2,2
2015,159,2,1,4,2
2016,156,2,2,4,4
2017,139,0,0,4,2
2018,144,0,2,2,5
2019,154,1,0,0,4
2020,103,0,2,0,2
2021,99,0,3,1,0


In [8]:
nearby['all_time'] = 'all time'
nearby.groupby('all_time').agg(
    crashes=('incident_id', 'count')
    , fatalities=('fatality', 'sum')
    , sbi=('sbi', 'sum')
    , pedestrian_ind=('pedestrian_ind', 'sum')
    , bicycle_ind=('bicycle_ind', 'sum')
).astype(int)

,crashes,fatalities,sbi,pedestrian_ind,bicycle_ind
all_time,,,,,
all time,1612,6,19,25,33


In [9]:
display_df = nearby.copy()
display_df['bicycle'] = display_df['bicycle_ind'].replace(0, '').replace(1, True)
display_df['pedestrian'] = display_df['pedestrian_ind'].replace(0, '').replace(1, True)
display_df['serious_injury'] = display_df['sbi'].replace(False, '')
display_df['fatal'] = display_df['fatality'].replace(False, '')

display_cols = [
    'reported_date_local'
    , 'top_traffic_accident_offense'
    , 'incident_address_corrected'
    , 'bicycle'
    , 'pedestrian'
    , 'serious_injury'
    , 'fatal'
    , 'distance_feet'
    , 'incident_id'
]

# nearby.to_clipboard()
# nearby[display_cols]
# display_df.loc[display_df.pedestrian_ind == 1, display_cols]
display_df[display_cols].tail(10)

,reported_date_local,top_traffic_accident_offense,incident_address_corrected,bicycle,pedestrian,serious_injury,fatal,distance_feet,incident_id
1602,2025-02-20 18:17:00-07:00,TRAF - ACCIDENT,2400 E 2ND AVE,,,,,2469.970134,202588431
1603,2025-02-21 11:28:00-07:00,TRAF - ACCIDENT,200 BLOCK N JOSEPHINE ST,,,,,2487.720904,202589704
1604,2025-02-28 13:33:00-07:00,TRAF - ACCIDENT,2300 BLOCK E 1ST AVE,,,,,2571.426716,202510339
1605,2025-03-10 11:16:00-06:00,TRAF - ACCIDENT,1200 BLOCK E 4TH AVE,,,,,2132.670883,2025121503
1606,2025-03-11 16:59:00-06:00,TRAF - ACCIDENT - HIT & RUN,E 1ST AVE / N LAFAYETTE ST,,,,,1098.123347,2025124199
1607,2025-03-13 14:39:00-06:00,TRAF - ACCIDENT - HIT & RUN,N UNIVERSITY BLVD / E 1ST AVE,,,,,2098.994383,2025127949
1608,2025-03-16 14:38:00-06:00,TRAF - ACCIDENT,303 N JOSEPHINE ST,,,,,2526.731019,2025133509
1609,2025-03-20 12:41:00-06:00,TRAF - ACCIDENT - HIT & RUN,N DOWNING ST / E 1ST AVE,,,,,1842.366117,2025140787
1610,2025-03-21 04:07:00-06:00,TRAF - ACCIDENT - HIT & RUN,200 BLOCK S HUMBOLDT ST,,,,,2429.951861,2025142059
1611,2025-03-23 20:30:00-06:00,TRAF - ACCIDENT - HIT & RUN,E SPEER BLVD / N DOWNING ST,,,,,2034.206363,2025147040


In [10]:
this_map = folium.Map(prefer_canvas=True)
HeatMap(nearby[['geo_lat', 'geo_lon']]).add_to(this_map)
folium.Circle(location=lat_lon, weight=2, radius=radius_miles*1609.34).add_to(this_map)
this_map.fit_bounds(
    [nearby[['geo_lat', 'geo_lon']].min().values.tolist()
     , nearby[['geo_lat', 'geo_lon']].max().values.tolist()]
)
this_map

In [13]:
# Text string with frequency of crashes
num_years = 3

most_recent_timestamp = cda.most_recent_crash_timestamp()
start_timestamp = most_recent_timestamp - pd.DateOffset(years=num_years)
days_in_time_range = (most_recent_timestamp - start_timestamp).days

nearby_in_time_range = nearby[nearby.reported_date_local > start_timestamp].copy()

days_between_crashes = days_in_time_range / len(nearby_in_time_range)

print(
    f'The location "{geocoded_location}" has had {len(nearby_in_time_range)} traffic crashes'
    , f'within a {radius_miles} mile radius'
    , f'over the last {num_years} years, an average of {days_between_crashes:.1f} days between crashes.'
    , f'In that time, there were {nearby_in_time_range.fatality.sum()} deadly crashes'
    , f'and {nearby_in_time_range.sbi.sum()} crashes causing a serious bodily injury.'
    , f'There were {(nearby_in_time_range.pedestrian_ind > 0).sum()} crashes involving a pedestrian'
    , f'and {(nearby_in_time_range.bicycle_ind > 0).sum()} crashes involving a bicyclist.'
)

The location "E 1st Ave and N Gilpin St, Denver, CO" has had 340 traffic crashes within a 0.5 mile radius over the last 3 years, an average of 3.2 days between crashes. In that time, there were 0 deadly crashes and 8 crashes causing a serious bodily injury. There were 4 crashes involving a pedestrian and 10 crashes involving a bicyclist.


In [14]:
cda.conn.dispose()

## Bike and ped crashes in nearby dataframe

In [17]:
nearby_in_time_range[nearby_in_time_range.bicycle_ind > 0]

,incident_id,top_traffic_accident_offense,reported_date,incident_address_corrected,at_freeway,geo_lon,geo_lat,neighborhood_id,bicycle_ind,pedestrian_ind,...,crash_date,crash_date_str,crash_time_str,crash_year,crash_day_of_year,geom,geom_denver,reported_date_local,distance_feet,all_time
1315,2022459508,TRAF - ACCIDENT,2022-09-06 16:00:00+00:00,E BAYAUD AVE / S CORONA ST,False,-104.974573,39.714721,Speer,1.0,0.0,...,2022-09-06,2022-09-06 Tue,"Tue Sep 6, 10:00 AM",2022,249,0101000020E61000004FC67C695F3E5AC05BBA7AFC7BDB...,0101000020AE0D0000D3D7F7FF1204484177613D008CB8...,2022-09-06 10:00:00-06:00,2564.885883,all time
1420,2023371555,TRAF - ACCIDENT,2023-07-12 00:15:00+00:00,S DOWNING ST / S MARION ST,False,-104.973148,39.714729,Washington Park,1.0,0.0,...,2023-07-11,2023-07-11 Tue,"Tue Jul 11, 6:15 PM",2023,192,0101000020E61000008547970E483E5AC097BB223A7CDB...,0101000020AE0D0000A977CF7FDB0448419AB9310091B8...,2023-07-11 18:15:00-06:00,2225.392172,all time
1433,2023431367,TRAF - ACCIDENT,2023-08-11 16:05:00+00:00,300 BLOCK N GILPIN ST,False,-104.966269,39.720411,Country Club,1.0,0.0,...,2023-08-11,2023-08-11 Fri,"Fri Aug 11, 10:05 AM",2023,223,0101000020E610000039626F5AD73D5AC0C0F8B36936DC...,0101000020AE0D00003FD3DBFF9C0848410796C0FFB1C0...,2023-08-11 10:05:00-06:00,795.760332,all time
1444,2023492164,TRAF - ACCIDENT - SBI,2023-09-12 13:37:00+00:00,E 1ST AVE / N DOWNING ST,False,-104.973232,39.718196,Speer,1.0,0.0,...,2023-09-12,2023-09-12 Tue,"Tue Sep 12, 7:37 AM",2023,255,0101000020E61000002B64956F493E5AC04BD424DBEDDB...,0101000020AE0D00007EFFCBFFCB0448415F5FF5FF7FBD...,2023-09-12 07:37:00-06:00,1842.366117,all time
1454,2023532507,TRAF - ACCIDENT,2023-10-03 02:24:00+00:00,E SPEER BLVD / N DOWNING ST,False,-104.973914,39.718334,Speer,1.0,0.0,...,2023-10-02,2023-10-02 Mon,"Mon Oct 2, 8:24 PM",2023,275,0101000020E6100000C071009A543E5AC09C272A5DF2DB...,0101000020AE0D0000CA32DEFF6B04484153D60B00B1BD...,2023-10-02 20:24:00-06:00,2034.206363,all time
1460,2023564262,TRAF - ACCIDENT - HIT & RUN,2023-10-19 05:09:00+00:00,N UNIVERSITY BLVD / E 1ST AVE,False,-104.959222,39.718275,Cherry Creek,1.0,0.0,...,2023-10-18,2023-10-18 Wed,"Wed Oct 18, 11:09 PM",2023,291,0101000020E61000002C1A00E4633D5AC0A3955A73F0DB...,0101000020AE0D00007A76CF7F7E0C48414D16B1FFB3BD...,2023-10-18 23:09:00-06:00,2098.994383,all time
1507,2024204797,TRAF - ACCIDENT - SBI,2024-04-17 05:57:00+00:00,N UNIVERSITY BLVD / E 1ST AVE,False,-104.959222,39.718275,Cherry Creek,1.0,0.0,...,2024-04-16,2024-04-16 Tue,"Tue Apr 16, 11:57 PM",2024,107,0101000020E61000002C1A00E4633D5AC0A3955A73F0DB...,0101000020AE0D00007A76CF7F7E0C48414D16B1FFB3BD...,2024-04-16 23:57:00-06:00,2098.994383,all time
1512,2024222371,TRAF - ACCIDENT - HIT & RUN,2024-04-26 18:49:00+00:00,N DOWNING ST / E 3RD AVE,False,-104.972933,39.720106,Speer,1.0,0.0,...,2024-04-26,2024-04-26 Fri,"Fri Apr 26, 12:49 PM",2024,117,0101000020E61000003FC2938A443E5AC037282F6C2CDC...,0101000020AE0D0000D3770600F4044841D41D8FFF37C0...,2024-04-26 12:49:00-06:00,1883.725607,all time
1530,2024335202,TRAF - ACCIDENT,2024-06-23 22:28:00+00:00,N DOWNING ST / E SPEER BLVD,False,-104.972888,39.718620,Country Club,1.0,0.0,...,2024-06-23,2024-06-23 Sun,"Sun Jun 23, 4:28 PM",2024,175,0101000020E61000005748D0CA433E5AC044F2BEBFFBDB...,0101000020AE0D0000FDC21200FC0448414C2A38001BBE...,2024-06-23 16:28:00-06:00,1750.575706,all time
1534,2024369543,TRAF - ACCIDENT,2024-07-11 00:22:00+00:00,N DOWNING ST / E 1ST AVE,False,-104.973232,39.718196,Speer,1.0,0.0,...,2024-07-10,2024-07-10 Wed,"Wed Jul 10, 6:22 PM",2024,192,0101000020E61000002B64956F493E5AC04BD424DBEDDB...,0101000020AE0D00007EFFCBFFCB0448415F5FF5FF7FBD...,2024-07-10 18:22:00-06:00,1842.366117,all time


In [18]:
nearby_in_time_range[nearby_in_time_range.pedestrian_ind > 0]

,incident_id,top_traffic_accident_offense,reported_date,incident_address_corrected,at_freeway,geo_lon,geo_lat,neighborhood_id,bicycle_ind,pedestrian_ind,...,crash_date,crash_date_str,crash_time_str,crash_year,crash_day_of_year,geom,geom_denver,reported_date_local,distance_feet,all_time
1353,2022639830,TRAF - ACCIDENT,2022-12-14 22:02:00+00:00,E 3RD AVE / N UNIVERSITY BLVD,False,-104.959436,39.720981,Country Club,0.0,1.0,...,2022-12-14,2022-12-14 Wed,"Wed Dec 14, 3:02 PM",2022,348,0101000020E610000057E49D64673D5AC0C999601749DC...,0101000020AE0D0000BC3F1B805D0C48410E5713008DC1...,2022-12-14 15:02:00-07:00,2268.562222,all time
1370,202367770,TRAF - ACCIDENT,2023-02-07 16:04:00+00:00,N UNIVERSITY BLVD / E 2ND AVE,False,-104.959441,39.719421,Country Club,0.0,1.0,...,2023-02-07,2023-02-07 Tue,"Tue Feb 7, 9:04 AM",2023,38,0101000020E6100000DDF63579673D5AC0305905FF15DC...,0101000020AE0D000000E1EC7F5E0C48411574640055BF...,2023-02-07 09:04:00-07:00,2081.667482,all time
1372,202378144,TRAF - ACCIDENT - HIT & RUN,2023-02-12 20:48:00+00:00,E ELLSWORTH AVE / S OGDEN ST,False,-104.975754,39.716530,Speer,0.0,1.0,...,2023-02-12,2023-02-12 Sun,"Sun Feb 12, 1:48 PM",2023,43,0101000020E6100000818B5FC2723E5AC037D9CC42B7DB...,0101000020AE0D0000A12EE5FF6A034841413E9BFF1CBB...,2023-02-12 13:48:00-07:00,2627.541709,all time
1582,2024672178,TRAF - ACCIDENT - SBI,2024-12-14 01:26:00+00:00,E 3RD AVE / N UNIVERSITY BLVD,False,-104.959436,39.720981,Country Club,0.0,1.0,...,2024-12-13,2024-12-13 Fri,"Fri Dec 13, 6:26 PM",2024,348,0101000020E610000057E49D64673D5AC0C999601749DC...,0101000020AE0D0000BC3F1B805D0C48410E5713008DC1...,2024-12-13 18:26:00-07:00,2268.562222,all time
